In [136]:
from collections import deque
from builtins import object
import fnmatch
import re

nil = object()

class TrieNode(object):

    __slots__ = ['char', 'output', 'fail', 'children']

    def __init__(self, char):
    # Make empty Node
            self.char = char # character
            self.output = nil # an output function for this node
            self.fail = nil # fail link used by Aho-Corasick automaton
            self.children = {} # children


class Trie(object):


    def __init__(self, case_sensitive = False, wildcards = False):
    # Make empty Trie
        self.root = TrieNode('')
        self.case_sensitive = case_sensitive
        self.wildcards = wildcards

    def keys(self):
    # Get all keys in Trie
        for key, _ in self.items():
            yield key

    def items(self):
    # Returns Keys and values stored in Trie
        L = []
        def aux(node, s):
            s = s + node.char
            if node.output is not nil:
                L.append((s, node.output))

            for child in node.children.values():
                if child is not node:
                    aux(child, s)

        aux(self.root, '')
        return iter(L)


    def __len__(self):
    # Get length of Trie
        stack = deque()
        stack.append(self.root)
        n = 0
        while stack:
            node = stack.pop()
            if node.output is not nil:
                n += 1

            for child in node.children.values():
                stack.append(child)

        return n


    def add_word(self, word, value):
    # Add word to the Trie
        if not word:
            return
            
            
        if self.case_sensitive:
            node = self.root
            for c in word:
                try:
                    node = node.children[c]
                except KeyError:
                    n = TrieNode(c)
                    node.children[c] = n
                    node = n

            node.output = value
        else:
            word = word.lower()

            node = self.root
            for c in word:
                try:
                    node = node.children[c]
                except KeyError:
                    n = TrieNode(c)
                    node.children[c] = n
                    node = n

            node.output = value


    def clear(self):
        self.root = TrieNode('')

    def aho_auto(self):
    # Create Automaton for Aho-Corasick
        queue = deque()
        for i in range(256):
            c = chr(i)
            if c in self.root.children:
                node = self.root.children[c]
                node.fail = self.root
                queue.append(node)
            else:
                self.root.children[c] = self.root
        while queue:
            r = queue.popleft()
            for node in r.children.values():
                queue.append(node)
                state = r.fail
                while node.char not in state.children:
                        state = state.fail

                node.fail = state.children.get(node.char, self.root)

# Iter is what we're using to search, will return all words at a given position so if you have
# the word "hers" in your text at position (index) 3 and you're looking for "he" and "hers" 
# (aka they're in your keyword list) then this will return something like [3, ("he", "hers")]
    def find_all(self, string):
        state = self.root
        if self.wildcards:
            for index, c in enumerate(string):
                while c not in state.children:
                    if '*' in state.children:
                      c = '*'
                      break;
                    else:
                      state = state.fail

                state = state.children.get(c, self.root)

                tmp = state
                output = []
                while tmp is not nil:
                    if tmp.output is not nil:
                        output.append(tmp.output)
                    tmp = tmp.fail

                if output:
                    yield (index, output)
        else:
            for index, c in enumerate(string):
                while c not in state.children:
                    state = state.fail

                state = state.children.get(c, self.root)

                tmp = state
                output = []
                while tmp is not nil:
                    if tmp.output is not nil:
                        output.append(tmp.output)

                    tmp = tmp.fail

                if output:
                    yield (index, output)



# Iter long basically finds the string at each index but only returns the longest word
# so if you have "hers" and are looking for the words "he" and "hers", iter_long only
# return "hers" and will ignore "he"
    def find_longest(self, string):
        if not self.case_sensitive:
            string = string.lower()
            
        if self.wildcards:
            state = self.root
            last  = None

            index = 0
            while index < len(string):
                c = string[index]

                if c not in state.children:
                  c = '*'

                if c in state.children:
                    state = state.children[c]

                    if state.output is not nil:
                        last = (state.output, index)

                    index += 1
                else:
                    if last:
                        yield last

                        index = last[1] + 1
                        state = self.root
                        last  = None
                    else:
                        while c not in state.children:
                            state = state.fail
            if last:
                yield last
        
        
        else:
            state = self.root
            last  = None

            index = 0
            while index < len(string):
                c = string[index]

                if c in state.children:
                    state = state.children[c]

                    if state.output is not nil:
                        last = (state.output, index)

                    index += 1
                else:
                    if last:
                        yield last
                        index = last[1] + 1
                        state = self.root
                        last  = None
                    else:
                        while c not in state.children:
                            state = state.fail
            if last:
                yield last


if __name__ == '__main__':

    def test_func_rb():
        found = []
        not_found = []
        res_list = []
        real_words = []
        count = {}
        count_dict = {}
        
        with open("./data/rainbw.txt", 'r') as text:
            s = text.read()

        words = "m*n source Saltwater sm*ller refract c*l*r glass Rainbow headphones jacket".split()
        

        trie = Trie(case_sensitive = True, wildcards = False)
        print("[Key Words]")
        for w in words:
            trie.add_word(w, w)
            count[w] = 0
            print(w)

        trie.aho_auto()

        print()
        print("Words Found (iter_long): ")
        
        s = re.sub('[^A-Za-z0-9\']+', '', s)
        s = s.replace(" ", "")
        for res in trie.find_longest(s):
            pos, matches = res
            count[res[0]] += 1
            found.append(res[0])
            res_list.append(res)
            
        
        for res in res_list:
            index = res[1]
            length = len(res[0])
            word=[]
            for i in range(index, (index-length), -1):
                word.append(s[i])
            word = word[::-1]
            string = ''.join(word)            
            real_words.append(string)

        not_found = set(words)-set(found)

        print()
        print("Words not found: ")
        if not not_found:
            print("All words found!")
        else:
            for n in not_found:
                print(n)
        
        print()
        print("[number of mentions]")
        for k, v in count.items():
            print(k,':', v)
        print()
        print("List of all words found (unique): ")
        for w in set(real_words):
            print(w)
        
        trie.clear()
  
    test_func_rb()



[Key Words]
m*n
source
Saltwater
antisolar
Moon
sm*ller
refract
color
glass
headphones
jacket

Words Found (iter_long): 

Words not found: 
glass
m*n
headphones
sm*ller
jacket

[number of mentions]
m*n : 0
source : 2
Saltwater : 1
antisolar : 4
Moon : 4
sm*ller : 0
refract : 11
color : 9
glass : 0
headphones : 0
jacket : 0

List of all words found (unique): 
source
refract
Moon
antisolar
Saltwater
color


In [143]:
    def test_func_hp():
        found = []
        not_found = []
        res_list = []
        real_words = []
        count = {}
        count_dict = {}
        
        with open("./data/HP_1.txt", 'r') as text:
            s = text.read()

        words = "iphone CAR Potter Dudley Privet keyboard scratching algorithm Draconis op*nion m*n V*rn*n freak".split()
        

        trie = Trie(case_sensitive = True, wildcards = True)
        print("[Key Words]")
        for w in words:
            trie.add_word(w, w)
            count[w] = 0
            print(w)

        trie.aho_auto()

        print()
        print("Words Found (iter_long): ")
        s = re.sub('[^A-Za-z0-9\']+', '', s)
        s = s.replace(" ", "")
        for res in trie.find_longest(s):
            pos, matches = res
            count[res[0]] += 1
            found.append(res[0])
            res_list.append(res)
            
        
        for res in res_list:
            index = res[1]
            length = len(res[0])
            word=[]
            for i in range(index, (index-length), -1):
                word.append(s[i])
            word = word[::-1]
            string = ''.join(word)            
            real_words.append(string)

        not_found = set(words)-set(found)

        print()
        print("Words not found: ")
        if not not_found:
            print("All words found!")
        else:
            for n in not_found:
                print(n)
        
        print()
        print("[number of mentions]")
        for k, v in count.items():
            print(k,':', v)
        print()
        print("List of all words found (unique): ")
        for w in set(real_words):
            print(w)
              
        trie.clear()
  
    test_func_hp()



[Key Words]
iphone
CAR
Potter
Dudley
Privet
keyboard
scratching
algorithm
Draconis
op*nion
m*n
V*rn*n
freak

Words Found (iter_long): 

Words not found: 
iphone
algorithm
keyboard

[number of mentions]
iphone : 0
CAR : 1
Potter : 132
Dudley : 137
Privet : 16
keyboard : 0
scratching : 3
algorithm : 0
Draconis : 1
op*nion : 5
m*n : 739
V*rn*n : 116
freak : 1

List of all words found (unique): 
freak
myn
mkn
men
min
CAR
Draconis
man
mun
msn
mon
mIn
Potter
scratching
Vernon
mUn
opinion
Privet
mOn
mAn
mEn
Dudley
